In [1]:
import os
import cv2
import numpy as np
import torch
from PIL import Image
from torchvision import transforms
from matplotlib import pyplot as plt
from tensorflow import keras, convert_to_tensor, reshape
from Lenet import LeNet
import pandas as pd

2023-03-07 17:48:54.213365: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-07 17:48:54.755054: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-07 17:48:54.755108: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-07 17:48:54.755112: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
label_dict = pd.read_csv('label_dict.csv')
label_dict_inv = pd.read_csv('label_dict_inv.csv')

In [3]:
def preprocess_keras(img):
#     cv2.imshow("input",img)
#     cv2.waitKey(0)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_inv = cv2.bitwise_not(gray)
    _, binary = cv2.threshold(gray_inv, 35, 255, cv2.THRESH_BINARY)
    # get the indices of the black pixels
    y, x = np.nonzero(binary)
    # get the maximum and minimum x and y coordinates
    max_x = np.max(x)
    min_x = np.min(x)
    max_y = np.max(y)
    min_y = np.min(y)
    nonzero_indices = np.nonzero(binary)
    zero_indices = np.where(binary == 0)
    darkened_gray = gray
#     darkened_gray = gray.astype('float32')
#     darkened_gray[zero_indices] = 229+0.1*darkened_gray[zero_indices]
#     darkened_gray[nonzero_indices] *= 0.1
#     darkened_gray = darkened_gray.astype('uint8')
    cropped = darkened_gray[min_y:max_y, min_x:max_x]
    # _, thresh_img = cv2.threshold(gray_roi, 155, 255, cv2.THRESH_BINARY)
#     inverted_img = cv2.bitwise_not(cropped)
    inverted_img = cropped
    aspect_ratio = inverted_img.shape[1] / inverted_img.shape[0]
    # resize the image while keeping the aspect ratio
    new_height = 20
    new_width = int(new_height * aspect_ratio)
    if new_width > new_height:
        new_width = 20
        aspect_ratio = inverted_img.shape[0] / inverted_img.shape[1]
        new_height = int(new_width * aspect_ratio)
    resized_img = cv2.resize(inverted_img, (new_width, new_height))
    # pad the resized image with white pixels to make it 28x28
    pad_width = (
        ((28 - new_height) // 2, (28 - new_height + 1) // 2),  # no padding on top and bottom
        ((28 - new_width) // 2, (28 - new_width + 1) // 2))  # pad equally on both sides to make the width 28
    padded_img = np.pad(resized_img, pad_width, mode='constant', constant_values=255)
    normalized_img = (cv2.resize(padded_img, (28, 28)))
#     cv2.imshow("preprocessed image", normalized_img)
#     cv2.waitKey(0)
    img_tensor = convert_to_tensor(normalized_img)
    img_tensor /= 255
    img_tensor = reshape(img_tensor, [1, 28, 28, 1])
    img_tensor = np.array(img_tensor)
    cv2.destroyAllWindows()
    return img_tensor


def keras_preprocessing(img):
    RGB = 1
    img = img.reshape(img.shape[0], img.shape[1], img.shape[2], RGB)
    img = img/255
    return img

In [4]:
def extract_characters(img_name, model):
    img = cv2.imread(img_name)
    
    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding to get a binary image
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 9, 5)

    # Find contours in the binary image
    contours, hierarchy = cv2.findContours(thresh, cv2.CHAIN_APPROX_SIMPLE, cv2.CHAIN_APPROX_SIMPLE)
    fields_points_sorted = []
    # Loop through the contours
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        threshold_value = 200
        black_found = False
        character = img[max((y + 3), 0):y + h - 3, max((x + 3), 0):x + w - 3]
        # Iterate over all pixels and check if their intensity values are above the threshold
        for row in range(character.shape[0]):
            for col in range(character.shape[1]):
                pixel_intensity = character[row, col]
                if (pixel_intensity < threshold_value).any():
                    black_found = True
                    break
            if black_found:
                break
        # Get the bounding rectangle of the contour
        if 30 < w < 100 and 40 < h < 100 and black_found:
            fields_points_sorted.append([max((y + 3), 0), y + h - 3, max((x + 3), 0), x + w - 3])
            # cv2.imwrite("C:/Users/30698/Desktop/KULeuven/Capita Selecta/test.jpeg", character)
            # show_one_prediction(character,model)
            #cv2.rectangle(img, (x + 3, y + 3), (x + w - 3, y + h - 3), (0, 255, 0), 2)
    fields_points_sorted.sort(key=lambda r: r[2])
    characters_sorted = []
    for field_points in fields_points_sorted:
        characters_sorted.append(img[field_points[0]:field_points[1], field_points[2]:field_points[3]])

    # Display the image with bounding boxes
#     cv2.imshow('image', characters_sorted[4])
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    return characters_sorted


def show_one_prediction(img, model, i=0):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_inv = cv2.bitwise_not(gray)
    _, binary = cv2.threshold(gray_inv, 25, 255, cv2.THRESH_BINARY)
    # get the indices of the black pixels
    y, x = np.nonzero(binary)

    # get the maximum and minimum x and y coordinates
    max_x = np.max(x)
    min_x = np.min(x)
    max_y = np.max(y)
    min_y = np.min(y)
    nonzero_indices = np.nonzero(binary)
    darkened_gray = gray.astype('float32')
    darkened_gray[nonzero_indices] *= 0.3
    darkened_gray[not nonzero_indices] *= 1.1
    darkened_gray = darkened_gray.astype('uint8')
    cropped = darkened_gray[min_y:max_y, min_x:max_x]
    cv2.imshow("bin", cropped)
    cv2.waitKey(0)
    # _, thresh_img = cv2.threshold(gray_roi, 155, 255, cv2.THRESH_BINARY)
    inverted_img = cv2.bitwise_not(cropped)
    aspect_ratio = inverted_img.shape[1] / inverted_img.shape[0]

    # resize the image while keeping the aspect ratio
    new_height = 24
    new_width = int(new_height * aspect_ratio)
    if new_width > new_height:
        return
    resized_img = cv2.resize(inverted_img, (new_width, new_height))

    # pad the resized image with black pixels to make it 28x28
    pad_width = (
        ((28 - new_height) // 2, (28 - new_height + 1) // 2),  # no padding on top and bottom
        ((28 - new_width) // 2, (28 - new_width + 1) // 2))  # pad equally on both sides to make the width 28
    padded_img = np.pad(resized_img, pad_width, mode='constant', constant_values=0)
    # img_brighter = cv2.add(inverted_img, 50)
    normalized_img = (cv2.resize(padded_img, (28, 28)))
    # Turn off gradients to speed up this part
    img_tensor = torch.from_numpy(normalized_img).unsqueeze(0)
    normalized_tensor = 2 * (img_tensor - img_tensor.min()) / (img_tensor.max() - img_tensor.min()) - 1
    # Define a transform to normalize the data
    # Apply the transform to the tensor
    print(normalized_tensor)
    with torch.no_grad():
        logps = model(normalized_tensor)

    # Output of the network are log-probabilities, need to take exponential for probabilities
    ps = torch.exp(logps)
    probab = list(ps.numpy()[0])
    # print(("pred:", probab.index(max(probab))))
    # print(("pred:", ))
    view_classify(img_tensor, ps)
    cv2.imshow("actual", img)
    cv2.waitKey(0)


def view_classify(img, ps):
    ''' Function for viewing an image and it's predicted classes.
    '''
    ps = ps.data.numpy().squeeze()

    fig, (ax1, ax2) = plt.subplots(figsize=(6, 9), ncols=2)
    ax1.imshow(img.numpy().squeeze(), cmap='gray_r')
    ax1.axis('off')
    ax2.barh(np.arange(10), ps)
    ax2.set_aspect(0.1)
    ax2.set_yticks(np.arange(10))
    ax2.set_yticklabels(np.arange(10))
    ax2.set_title('Class Probability')
    ax2.set_xlim(0, 1.1)
    plt.tight_layout()
    plt.show()


In [5]:
people = [
    ['MICHELLE', 'RODRIGUEZ', '12/02/1950', 'RFMR12025033445', 'DE1202507171'],
    ['SIERRA', 'YU', '29/03/1950', 'RFSY29035047838', 'IT2903504950'],
    ['LAURA', 'MENDOZA', '16/10/1952', 'RFLM16105293143', 'BR1610522746'],
    ['ANGELA', 'FITZGERALD', '04/07/1956', 'RFAF04075656622', 'BE0407563203'],
    ['DAVID', 'LOPEZ', '17/01/1963', 'RFDL17016329220', 'BR1701638007'],
    ['HECTOR', 'COOK', '09/02/1964', 'RFHC09026495520', 'IT0902647355'],
    ['TIMOTHY', 'GREEN', '19/08/1968', 'RFTG19086827450', 'BR1908688556'],
    ['ELIZABETH', 'RIVERA', '08/07/1970', 'RFER08077010123', 'BR0807708065'],
    ['DEREK', 'CHURCH', '13/01/1972', 'RFDC13017297192', 'BL1301728110'],
    ['JEREMIAH', 'TUCKER', '11/10/1972', 'RFJT11107251133', 'BR1110727764'],
    ['LEAH', 'FERGUSON', '05/01/1976', 'RFLF05017691772', 'US0501769434'],
    ['KENNETH', 'STONE', '19/01/1977', 'RFKS19017711799', 'DE1901777829'],
    ['EDWIN', 'ALLEN', '22/09/1977', 'RFEA22097775609', 'DE2209776709'],
    ['ANGEL', 'SCHWARTZ', '18/05/1982', 'RFAS18058263146', 'BR1805822377'],
    ['JARED', 'MURRAY', '26/01/1985', 'RFJM26018598155', 'BR2601853435'],
    ['MELISSA', 'MENDOZA', '11/06/1987', 'RFMM11068724346', 'BL1106877773'],
    ['NICOLE', 'CANTU', '19/03/1989', 'RFNC19038962968', 'DE1903891767'],
    ['JAMES', 'HARRISON', '17/02/1990', 'RFJH17029026206', 'BE1702909215'],
    ['JEFFREY', 'MOONEY', '19/04/1990', 'RFJM19049013684', 'BE1904901793'],
    ['LORI', 'NUNEZ', '28/06/1997', 'RFLN28069748554', 'BR2806974343']

]


true_labels=[
    ['NICOLE', 'CANTU', '19/03/1989', 'RFNC19038962968', 'DE1903891767'],
    ['JEREMIAH', 'TUCKER', '11/10/1972', 'RFJT11107251133', 'BR1110727764'],
    ['LORI', 'NUNEZ', '28/06/1997', 'RFLN28069748554', 'BR2806974343'],
    ['SIERRA', 'YU', '29/03/1950', 'RFSY29035047838', 'IT2903504950'],
    ['TIMOTHY', 'GREEN', '19/08/1968', 'RFTG19086827450', 'BR1908688556'],
    ['LAURA', 'MENDOZA', '16/10/1952', 'RFLM16105293143', 'BR1610522746'],
    ['JAMES', 'HARRISON', '17/02/1990', 'RFJH17029026206', 'BE1702909215'],
    ['SIERRA', 'YU', '29/03/1950', 'RFSY29035047838', 'IT2903504950'],
    ['ELIZABETH', 'RIVERA', '08/07/1970', 'RFER08077010123', 'BR0807708065'],
    ['NICOLE', 'CANTU', '19/03/1989', 'RFNC19038962968', 'DE1903891767'],
    ['KENNETH', 'STONE', '19/01/1977', 'RFKS19017711799', 'DE1901777829'],
    ['MICHELLE', 'RODRIGUEZ', '12/02/1950', 'RFMR12025033445', 'DE1202507171'],
    ['DEREK', 'CHURCH', '13/01/1972', 'RFDC13017297192', 'BL1301728110'],
    ['DAVID', 'LOPEZ', '17/01/1963', 'RFDL17016329220', 'BR1701638007'],
    ['JAMES', 'HARRISON', '17/02/1990', 'RFJH17029026206', 'BE1702909215'],
    ['JEREMIAH', 'TUCKER', '11/10/1972', 'RFJT11107251133', 'BR1110727764'],
    ['EDWIN', 'ALLEN', '22/09/1977', 'RFEA22097775609', 'DE2209776709'],
    ['ELIZABETH', 'RIVERA', '08/07/1970', 'RFER08077010123', 'BR0807708065'],
    ['MELISSA', 'MENDOZA', '11/06/1987', 'RFMM11068724346', 'BL1106877773'],
    ['JEREMIAH', 'TUCKER', '11/10/1972', 'RFJT11107251133', 'BR1110727764'],
    ['LORI', 'NUNEZ', '28/06/1997', 'RFLN28069748554', 'BR2806974343'],
    ['SIERRA', 'YU', '29/03/1950', 'RFSY29035047838', 'IT2903504950'],
    ['JAMES', 'HARRISON', '17/02/1990', 'RFJH17029026206', 'BE1702909215'],
    ['ANGEL', 'SCHWARTZ', '18/05/1982', 'RFAS18058263146', 'BR1805822377'],
    ['HECTOR', 'COOK', '09/02/1964', 'RFHC09026495520', 'IT0902647355'],
    ['LEAH', 'FERGUSON', '05/01/1976', 'RFLF05017691772', 'US0501769434'],
    ['JARED', 'MURRAY', '26/01/1985', 'RFJM26018598155', 'BR2601853435'],
    ['NICOLE', 'CANTU', '19/03/1989', 'RFNC19038962968', 'DE1903891767'],
    ['JEFFREY', 'MOONEY', '19/04/1990', 'RFJM19049013684', 'BE1904901793'],
    ['HECTOR', 'COOK', '09/02/1964', 'RFHC09026495520', 'IT0902647355'],
    ['SIERRA', 'YU', '29/03/1950', 'RFSY29035047838', 'IT2903504950'],
    ['LEAH', 'FERGUSON', '05/01/1976', 'RFLF05017691772', 'US0501769434'],
    ['TIMOTHY', 'GREEN', '19/08/1968', 'RFTG19086827450', 'BR1908688556'],
    ['LORI', 'NUNEZ', '28/06/1997', 'RFLN28069748554', 'BR2806974343'],
    ['LEAH', 'FERGUSON', '05/01/1976', 'RFLF05017691772', 'US0501769434'],
    ['JEREMIAH', 'TUCKER', '11/10/1972', 'RFJT11107251133', 'BR1110727764'],
    ['LAURA', 'MENDOZA', '16/10/1952', 'RFLM16105293143', 'BR1610522746'],
    ['ANGELA', 'FITZGERALD', '04/07/1956', 'RFAF04075656622', 'BE0407563203'],
    ['NICOLE', 'CANTU', '19/03/1989', 'RFNC19038962968', 'DE1903891767'],
    ['JEFFREY', 'MOONEY', '19/04/1990', 'RFJM19049013684', 'BE1904901793'],
    ['KENNETH', 'STONE', '19/01/1977', 'RFKS19017711799', 'DE1901777829'],
    ['JARED', 'MURRAY', '26/01/1985', 'RFJM26018598155', 'BR2601853435'],
    ['LORI', 'NUNEZ', '28/06/1997', 'RFLN28069748554', 'BR2806974343'],
    ['JEFFREY', 'MOONEY', '19/04/1990', 'RFJM19049013684', 'BE1904901793']
]



In [6]:
def predict_date(model, characters_for_recognition, labels):
    i=0
    correct_fields=0
    for char in labels:
        if char != '/':
            processed_input = preprocess_keras(characters_for_recognition[i])            
            pred_vector = model.predict(processed_input)
            print(f"Prediction Vector: {pred_vector}")
            argmax = str(np.argmax(pred_vector))
            pred = str(label_dict_inv[argmax].get(0))
            print(f"Prediction: {pred}")
            if pred == char:
                correct_fields+=1
        i += 1
    return correct_fields

In [7]:
# model_digits = LeNet()
# model_digits.load_state_dict(torch.load('saved_models/digits_model_nn3'))
model_digits = keras.models.load_model('./alphanum_model')

print("Model successfully loaded")

# extract_characters('C:/Users/30698/Desktop/KULeuven/Capita Selecta/formes/fields/0001_DATE.jpg')
# dir_path = 'C:/Users/30698/Desktop/KULeuven/Capita Selecta/formes/fione
fields_dir_path = "./Forms 24022023/fields/"

# Define the allowed image extensions
img_extensions = ["DATE.jpg", "DATE.jpeg", "DATE.png"]

2023-03-07 17:48:55.633364: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-07 17:48:55.638087: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-07 17:48:55.638466: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-07 17:48:55.639341: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

Model successfully loaded


In [8]:
for root, dirs, files in os.walk(fields_dir_path):
        # Iterate over all files in the current directory
        correct=0
        print(f"Found {len(files)} files in '{fields_dir_path}'")
        for file in files:
            # Check if the file has an image extension            
            if any(file.endswith(ext) for ext in img_extensions):
                print(f"Checking for {file}:")
                # Add the full path of the image to the list
                img_path = os.path.join(root, file)
                characters_of_date = extract_characters(img_path, model_digits)
                file_index = int(file[:4])
                print(file_index)
                correct += predict_date(model_digits, characters_of_date, true_labels[file_index-1][2])
        print("Accuracy:", correct/(44*8))

Found 220 files in './Forms 24022023/fields/'
Checking for 0017_DATE.jpg:
17


2023-03-07 17:48:58.287741: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8600


1/1 [==============================] - 1s 1s/step
Prediction Vector: [[5.29042794e-04 1.88267237e-04 2.70481032e-05 7.42735779e-07
  1.88027320e-06 9.03754626e-05 8.44446258e-05 3.21027637e-03
  3.65986081e-04 2.54788847e-05 2.37685876e-04 6.15393219e-05
  8.03094226e-05 7.19723292e-04 8.55042163e-06 3.14123163e-08
  9.93369240e-03 1.32405721e-02 2.54745712e-04 4.53272914e-05
  2.33310787e-03 6.16639376e-01 7.53894623e-04 1.25101060e-02
  1.96038522e-02 8.55096758e-08 3.11560859e-03 1.12795096e-03
  1.06049396e-04 1.13845393e-02 1.18792802e-03 2.48195221e-07
  7.85436854e-03 2.84861773e-04 5.73644938e-04 8.87821312e-04
  1.87222213e-05 2.17398429e-06 8.28137203e-08 8.90053434e-06
  9.62245940e-06 1.02965510e-03 1.25049637e-09 1.07761822e-03
  4.13218979e-03 3.73238508e-05 1.90090450e-08 6.13954151e-03
  2.92880200e-02 4.13323141e-06 3.26778973e-05 2.14126403e-03
  4.68434906e-03 5.93162731e-05 3.67800812e-09 1.57489067e-05
  5.63524445e-05 1.31626322e-07 4.19304882e-07 3.20150320e-05
 

2023-03-07 17:48:58.910136: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


Prediction Vector: [[5.7535464e-05 4.0399737e-12 3.8441311e-05 3.6248560e-09 6.8297595e-06
  4.7520066e-06 4.3867368e-07 8.5827480e-05 4.9060700e-06 6.4712340e-06
  3.4622428e-08 2.1765945e-06 3.5505087e-04 1.0299273e-03 3.0203887e-14
  3.0516660e-09 3.0470095e-04 8.8669062e-03 3.7424350e-06 1.0593236e-04
  1.6363231e-04 7.7582681e-01 8.5238702e-05 2.2497201e-05 2.4703268e-02
  2.5846116e-12 1.0424204e-09 2.3050692e-04 1.9032581e-04 1.6222659e-02
  3.5933615e-04 2.4861233e-08 2.2478651e-03 5.0044442e-03 6.5515082e-09
  1.4324441e-05 1.7427846e-09 1.4818132e-12 4.6903930e-09 3.4304264e-07
  5.3238605e-09 3.4864904e-06 5.5972987e-12 1.2116230e-05 1.4149876e-03
  3.5343010e-07 4.7348452e-11 2.7301130e-06 4.1637099e-03 4.7432030e-07
  3.5178857e-06 2.9950459e-07 8.4788527e-04 2.9334676e-06 4.3465943e-12
  1.1054938e-08 2.2003221e-04 1.3840626e-07 5.5361926e-07 2.6574904e-05
  4.2299642e-03 1.0501233e-09 2.4250562e-06 3.2999105e-04 2.7637982e-03
  1.3579393e-04 1.8419216e-04 3.4477771e-03 4

...

1/1 [==============================] - 0s 13ms/step
Prediction Vector: [[1.89346065e-05 6.35159503e-12 2.84506677e-06 4.24025837e-09
  4.37689579e-07 6.45985074e-06 1.37465645e-06 6.49633512e-05
  4.43189667e-04 7.31819682e-06 6.50426045e-06 7.34301807e-07
  8.05643387e-03 1.17707113e-03 1.20344800e-16 6.77877761e-05
  1.28389988e-03 2.11277953e-03 2.77345625e-05 5.04556556e-05
  3.29689537e-05 9.16429877e-01 2.35940839e-04 3.34907891e-05
  7.52198335e-04 5.81928386e-13 2.62415735e-04 3.37461068e-04
  5.40474730e-06 2.70159468e-02 1.45569209e-06 9.29068975e-11
  1.48962974e-03 1.24397238e-05 8.90364060e-10 1.36301992e-06
  1.14817517e-10 6.16411653e-13 4.12549355e-08 5.29726606e-07
  1.34450646e-08 2.36384280e-04 3.57467633e-09 7.49650962e-06
  1.23569393e-04 1.66168866e-06 8.63622862e-09 3.88889719e-04
  4.49350511e-04 1.85738480e-09 1.35112314e-05 1.04070546e-06
  4.98469308e-06 2.40177519e-06 1.85485335e-13 1.47961671e-07
  3.63289868e-03 1.61993484e-07 1.29809166e-07 3.21649100e-06

1/1 [==============================] - 0s 12ms/step
Prediction Vector: [[2.4713543e-05 7.1753899e-04 7.6055733e-07 7.6795142e-05 3.5312357e-08
  5.5989938e-05 8.0783051e-05 1.7944396e-06 5.0654955e-05 6.2289395e-08
  5.3297265e-07 1.8795597e-04 2.1162417e-03 1.1079186e-05 4.3054098e-09
  3.4956269e-05 2.1383492e-02 2.7588243e-04 1.3779250e-05 4.2075393e-04
  2.1726141e-06 6.8882734e-01 1.4713027e-06 3.6759358e-03 3.4458492e-02
  1.0509959e-07 6.0800821e-06 7.5035226e-03 1.2433015e-06 5.1172357e-03
  2.1822858e-05 2.3159748e-05 1.1348553e-05 1.9885731e-05 3.5571892e-07
  1.0160464e-03 2.3114546e-06 9.6059591e-07 1.0758666e-06 1.2953232e-04
  1.2623212e-07 2.6516291e-05 2.8019555e-05 2.0445239e-04 2.2753455e-04
  8.4545107e-05 1.5190061e-06 5.2762567e-03 2.0935931e-04 6.0599283e-05
  2.0057529e-03 7.1711896e-04 8.5577676e-07 2.0798470e-03 1.7039696e-08
  3.5146857e-06 1.0279650e-02 1.9953035e-05 2.2519613e-08 1.2755531e-05
  5.4484194e-06 8.2427119e-05 2.5379305e-07 5.4650463e-06 4.95396

  1.22858004e-14 4.59506966e-09]]
Prediction: "
1/1 [==============================] - 0s 13ms/step
Prediction Vector: [[2.7353084e-09 1.3930581e-11 3.0880745e-09 5.4664728e-11 3.1603700e-10
  1.5857187e-06 2.1629726e-06 2.1848539e-09 1.3578416e-05 7.3032598e-09
  6.2757849e-08 1.6057791e-06 2.2490190e-02 1.1651616e-06 3.5242333e-15
  1.3481751e-04 1.9740914e-03 8.8411610e-04 6.0946209e-06 1.2697323e-07
  6.1764877e-07 9.5321733e-01 8.8488150e-06 1.3562018e-07 3.1893430e-04
  3.4294922e-12 1.0413375e-05 3.6187565e-05 4.4630055e-09 9.3701305e-03
  3.4921825e-07 5.2970259e-14 6.7295387e-06 4.2671467e-11 6.4419112e-12
  3.2557100e-08 2.0225721e-09 1.4667622e-14 8.5426271e-09 1.2759560e-09
  5.0647841e-10 1.9681895e-05 2.0817636e-08 1.2202576e-05 5.9478652e-06
  7.9498761e-08 2.1417462e-08 5.6580157e-06 5.8678029e-07 8.2841763e-12
  1.1784240e-06 8.2348922e-10 2.9369767e-09 6.8889642e-07 9.0259614e-13
  3.1851950e-09 9.8829984e-04 4.6417030e-09 4.3801861e-11 7.3596934e-10
  9.0268109e-08 9

1/1 [==============================] - 0s 13ms/step
Prediction Vector: [[9.07988849e-08 1.36951328e-14 1.49763384e-08 2.47938844e-12
  2.76329915e-09 5.87324280e-07 6.02105743e-08 8.31666256e-08
  5.06185934e-05 1.12110916e-08 2.24116761e-08 1.19486131e-07
  7.61730038e-03 7.74086311e-06 1.60036668e-17 7.49423498e-06
  2.03216638e-04 2.56416807e-03 1.50489427e-06 7.45676061e-07
  2.34387153e-05 9.71105814e-01 4.01615989e-06 8.85451001e-08
  1.63621665e-03 5.54314454e-16 5.43312053e-05 7.98820020e-06
  2.25411014e-08 5.08619240e-03 2.57198985e-06 4.78971619e-11
  3.17498634e-05 1.89339957e-08 1.77203347e-12 1.70680119e-06
  5.29449679e-11 8.71763027e-16 1.12170750e-09 4.94091268e-10
  6.52262272e-11 4.17531955e-05 5.49812279e-11 1.15961404e-06
  6.46991748e-06 4.10298284e-08 1.80425362e-11 2.10956887e-06
  6.19813582e-05 2.28247687e-09 9.59832605e-08 4.72839434e-10
  4.57644838e-08 9.20906018e-08 3.06183405e-16 7.03470882e-09
  1.65984966e-03 1.30675459e-09 2.83148838e-10 1.58860427e-08

1/1 [==============================] - 0s 12ms/step
Prediction Vector: [[3.2261730e-07 1.9370544e-22 6.6226980e-06 1.1320385e-09 2.0056548e-06
  4.0165555e-06 1.0436803e-06 1.4465971e-08 2.7493921e-03 2.8469588e-06
  6.3773032e-13 2.2553536e-06 2.0536770e-01 4.3583106e-05 6.3996922e-19
  3.9097372e-06 1.0284000e-05 1.0218504e-02 6.5774802e-06 3.1081388e-05
  8.3671315e-03 4.7365543e-01 3.3275937e-06 8.3058397e-13 3.7790392e-02
  7.0662707e-15 3.0772924e-06 1.3800658e-04 7.3511677e-07 1.1801195e-01
  4.8160371e-05 1.3598564e-09 1.4393423e-04 7.1770580e-07 3.7948829e-15
  3.1461266e-05 6.4199673e-10 1.9744886e-17 1.2167921e-07 3.8812999e-07
  5.9807044e-09 1.5356710e-04 4.3543114e-10 2.9103837e-06 2.5519448e-05
  4.0371201e-08 1.1621034e-11 3.4747561e-08 7.7199808e-04 1.7890335e-08
  2.4313690e-06 3.1416038e-17 3.1379627e-06 3.9859146e-06 1.0484191e-13
  1.1856089e-08 5.4436434e-02 3.9753965e-07 9.8065823e-07 4.0766702e-07
  1.0214275e-04 9.4564512e-12 1.8798701e-06 2.3210509e-04 9.16289

In [9]:
cv2.destroyAllWindows()